In [1]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multitest import multipletests

In [2]:
df = pd.read_csv('Results_TD_CP.csv')

metrics = ['dti_fa', 'md', 'ad', 'rd']

In [3]:
comparisons = [
    # Wilcoxon
    ('CSC_LA', 'CSC_MA', 'wilcoxon'),
    ('PV_LA',  'PV_MA',  'wilcoxon'),
    
    # Mann-Whitney
    ('CSC_LA', 'TD_D',   'mannwhitney'),
    ('PV_LA',  'TD_D',   'mannwhitney'),
    ('PV_LA',  'CSC_LA', 'mannwhitney'),
    ('CSC_MA', 'TD_ND',  'mannwhitney'),
    ('PV_MA',  'TD_ND',  'mannwhitney'),
    ('PV_MA',  'CSC_MA', 'mannwhitney')
]

In [4]:
print(f"Targeted Analysis with FDR Correction (Benjamini-Hochberg):")
print(f"{'Metrics':<8} | {'Comparison':<18} | {'Test':<12} | {'Raw p':<10} | {'FDR p':<10} | {'Result'}")
#print("-" * 90)

for metric in metrics:
    subset = df[df['Metrics Name'] == metric] 
    
    p_values = []
    comp_labels = []
    test_types = []
    
    # Running tests and collecting raw p-values
    for g1, g2, test_type in comparisons:
        # Wilcoxon (Paired)
        if test_type == 'wilcoxon':
            paired_data = subset[[g1, g2]].dropna()
            if len(paired_data) < 3:
                p_val = 1.0
            else:
                stat, p_val = stats.wilcoxon(paired_data[g1], paired_data[g2])
        
        # Mann-Whitney (Independent)
        else:
            data1 = subset[g1].dropna()
            data2 = subset[g2].dropna()
            if len(data1) == 0 or len(data2) == 0:
                p_val = 1.0
            else:
                stat, p_val = stats.mannwhitneyu(data1, data2, alternative='two-sided')
        
        p_values.append(p_val)
        comp_labels.append(f"{g1} vs {g2}")
        test_types.append(test_type)
    
    # Applying FDR Correction
    reject, pvals_corrected, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')
    
    # Printing Results
    print(f"{metric}")
    for i in range(len(comparisons)):
        # If reject[i] is True, it means "Significant"
        result_text = "Significant" if reject[i] else "No"
        
        print(f"{'':<8} | {comp_labels[i]:<18} | {test_types[i]:<12} | {p_values[i]:.2e}   | {pvals_corrected[i]:.2e}   | {result_text}")
    print("")

Targeted Analysis with FDR Correction (Benjamini-Hochberg):
Metrics  | Comparison         | Test         | Raw p      | FDR p      | Result
dti_fa
         | CSC_LA vs CSC_MA   | wilcoxon     | 1.25e-01   | 2.50e-01   | No
         | PV_LA vs PV_MA     | wilcoxon     | 4.69e-02   | 1.25e-01   | No
         | CSC_LA vs TD_D     | mannwhitney  | 4.85e-01   | 6.46e-01   | No
         | PV_LA vs TD_D      | mannwhitney  | 7.46e-01   | 8.53e-01   | No
         | PV_LA vs CSC_LA    | mannwhitney  | 8.76e-01   | 8.76e-01   | No
         | CSC_MA vs TD_ND    | mannwhitney  | 2.97e-04   | 2.38e-03   | Significant
         | PV_MA vs TD_ND     | mannwhitney  | 9.33e-04   | 3.73e-03   | Significant
         | PV_MA vs CSC_MA    | mannwhitney  | 2.68e-01   | 4.28e-01   | No

md
         | CSC_LA vs CSC_MA   | wilcoxon     | 1.25e-01   | 2.50e-01   | No
         | PV_LA vs PV_MA     | wilcoxon     | 1.56e-02   | 6.25e-02   | No
         | CSC_LA vs TD_D     | mannwhitney  | 9.76e-01   | 9.76e-01   